# Credit Risk Ensemble Techniques

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [15]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [16]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

df.head()

df_clean = pd.get_dummies(df, columns=["homeowner"])
df_clean.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


# Split the Data into Training and Testing

In [21]:
# Create our features
X = df_clean.drop(columns="loan_status")

# Create our target
y = df_clean["loan_status"].ravel()

In [7]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [18]:
# Check the balance of our target values
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [19]:
# Split the X and y into X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)



In [46]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.fit_transform(X_test)



array([[ 0.14045627,  0.1230916 ,  0.12859477, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-0.28969527, -0.27393966, -0.27772578, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-0.48087373, -0.47526712, -0.48088605, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       ...,
       [-1.48456066, -1.46728289, -1.47278621, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-0.28969527, -0.26831599, -0.26577518, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-1.05440912, -1.05787955, -1.05451506, ...,  1.00731801,
        -0.81861018, -0.3377677 ]])

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [47]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_ss, y_train)


BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [49]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brf.predict(X_test_ss)

balanced_accuracy_score(y_test, y_pred)

0.9941883042806119

In [51]:
# Display the confusion matrix

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)



array([[  622,     3],
       [  128, 18631]])

In [53]:
# Print the imbalanced classification report

y_pred_ = brf.predict(X_test_ss)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      1.00      0.99      0.90      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [54]:
# List the features sorted in descending order by feature importance

imp = brf.feature_importances_

ind = np.argsort(imp)[::-1]

for i in range(X.shape[1]):
    print(f"{X.columns.values[ind[i]]}: ({imp[ind[i]]})")



interest_rate: (0.2105321509900115)
borrower_income: (0.20040413958739037)
debt_to_income: (0.16680831490340173)
total_debt: (0.16486313476058623)
loan_size: (0.1420448820333474)
num_of_accounts: (0.09282588861202129)
derogatory_marks: (0.020059024240050956)
homeowner_own: (0.0010293976962115277)
homeowner_mortgage: (0.0009307421234053509)
homeowner_rent: (0.0005023250535736838)


### Easy Ensemble Classifier

In [55]:
# Train the EasyEnsembleClassifier

from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(random_state=1)

eec.fit(X_train_ss, y_train)

y_pred = eec.predict(X_test_ss)





In [56]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.994748035609574

In [57]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)


array([[  622,     3],
       [  107, 18652]])

In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.91      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384

